In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install --upgrade unsloth
!pip install "transformers>=4.37.0" accelerate bitsandbytes datasets pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 2.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.8/298.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.7/164.7 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 132.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/2

In [3]:
from unsloth import FastLanguageModel
from transformers import AutoProcessor
from PIL import Image
import torch
import json
from datasets import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.optim import AdamW

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
# Chọn đúng base model bạn muốn train
base_model = "unsloth/Qwen2-VL-2B-Instruct"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=base_model,
    max_seq_length=2048,
    dtype="float16",        # hoặc "float32" nếu máy không hỗ trợ float16
    load_in_4bit=False      # KHÔNG dùng 4bit nếu không thực sự cần
)

processor = AutoProcessor.from_pretrained(base_model)

==((====))==  Unsloth 2025.7.5: Fast Qwen2 patching. Transformers: 4.53.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


model.safetensors:   0%|          | 0.00/4.42G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/572 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


chat_template.json: 0.00B [00:00, ?B/s]

In [5]:
# Đường dẫn tới file dữ liệu của bạn
data_path = "/content/rejected_dataset_dpo.json"

with open(data_path, "r", encoding="utf-8") as f:
    data = json.load(f)
dataset = Dataset.from_list(data)

In [6]:
class DPOCollatorWithImages:
    def __init__(self, processor):
        self.processor = processor

    def __call__(self, batch):
        images = [Image.open(ex["image_path"]).convert("RGB") for ex in batch]
        prompts = [ex["prompt"] for ex in batch]
        chosens = [ex["chosen"] for ex in batch]
        rejecteds = [ex["rejected"] for ex in batch]

        chosen_inputs = self.processor(
            images=images, text=[p + c for p, c in zip(prompts, chosens)],
            return_tensors="pt", padding=True, truncation=True
        )
        rejected_inputs = self.processor(
            images=images, text=[p + r for p, r in zip(prompts, rejecteds)],
            return_tensors="pt", padding=True, truncation=True
        )

        return {
            "chosen_input_ids": chosen_inputs["input_ids"],
            "chosen_attention_mask": chosen_inputs["attention_mask"],
            "rejected_input_ids": rejected_inputs["input_ids"],
            "rejected_attention_mask": rejected_inputs["attention_mask"],
        }

collator = DPOCollatorWithImages(processor)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collator)

In [7]:
optimizer = AdamW(model.parameters(), lr=5e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.train()

def compute_logprobs(logits, input_ids, attention_mask):
    log_probs = F.log_softmax(logits, dim=-1)
    token_logprobs = log_probs.gather(dim=-1, index=input_ids.unsqueeze(-1)).squeeze(-1)
    token_logprobs = token_logprobs * attention_mask
    return token_logprobs.sum(dim=-1) / attention_mask.sum(dim=-1)

num_epochs = 3
beta = 0.1

for epoch in range(num_epochs):
    for step, batch in enumerate(dataloader):
        for k in batch:
            batch[k] = batch[k].to(device)

        optimizer.zero_grad()

        chosen_logits = model(input_ids=batch["chosen_input_ids"],
                              attention_mask=batch["chosen_attention_mask"]).logits
        rejected_logits = model(input_ids=batch["rejected_input_ids"],
                                attention_mask=batch["rejected_attention_mask"]).logits

        chosen_score = compute_logprobs(chosen_logits, batch["chosen_input_ids"], batch["chosen_attention_mask"])
        rejected_score = compute_logprobs(rejected_logits, batch["rejected_input_ids"], batch["rejected_attention_mask"])

        loss = -F.logsigmoid(beta * (chosen_score - rejected_score)).mean()
        loss.backward()
        optimizer.step()

        if step % 10 == 0:
            print(f"Epoch {epoch}, Step {step}, Loss: {loss.item():.4f}")

    # Lưu mô hình sau mỗi epoch
    save_path = f"/content/qwen2vl_unsloth_epoch{epoch}"
    model.save_pretrained(save_path, safe_serialization=True)
    tokenizer.save_pretrained(save_path)
    print(f"✅ Saved model to {save_path}")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Epoch 0, Step 0, Loss: 0.7080
Epoch 0, Step 10, Loss: 0.7134
Epoch 0, Step 20, Loss: 0.7251
Epoch 0, Step 30, Loss: 0.7119
Epoch 0, Step 40, Loss: 0.7124
Epoch 0, Step 50, Loss: 0.7256
Epoch 0, Step 60, Loss: 0.7173
Epoch 0, Step 70, Loss: 0.7158
Epoch 0, Step 80, Loss: 0.7061
Epoch 0, Step 90, Loss: 0.7061
Epoch 0, Step 100, Loss: 0.7090
Epoch 0, Step 110, Loss: 0.7246
Epoch 0, Step 120, Loss: 0.7163
Epoch 0, Step 130, Loss: 0.7075
Epoch 0, Step 140, Loss: 0.7197
Epoch 0, Step 150, Loss: 0.7158
Epoch 0, Step 160, Loss: 0.7119
Epoch 0, Step 170, Loss: 0.7041
Epoch 0, Step 180, Loss: 0.7197
Epoch 0, Step 190, Loss: 0.7158
Epoch 0, Step 200, Loss: 0.7280
Epoch 0, Step 210, Loss: 0.7129
Epoch 0, Step 220, Loss: 0.7188
Epoch 0, Step 230, Loss: 0.7036
Epoch 0, Step 240, Loss: 0.7134
Epoch 0, Step 250, Loss: 0.7002
Epoch 0, Step 260, Loss: 0.7100
Epoch 0, Step 270, Loss: 0.7139
Epoch 0, Step 280, Loss: 0.7158
Epoch 0, Step 290, Loss: 0.7236
Epoch 0, Step 300, Loss: 0.7134
Epoch 0, Step 310, 

**Inference**

In [ ]:
from unsloth import FastLanguageModel
from transformers import AutoProcessor
from PIL import Image
import torch

ckpt_path = "/content/qwen2vl_unsloth_epoch2"  # Đường dẫn checkpoint đã lưu

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=ckpt_path,
    max_seq_length=2048,
    dtype="float16",
    load_in_4bit=False
)
processor = AutoProcessor.from_pretrained(ckpt_path)

image_path = "/content/drive/MyDrive/REL_Project/Data/test/images/aguanambi-2555_png_jpg.rf.e8fe552acc205329ac459d1087eb0637.jpg"
prompt = "<|user|>\nCan you tell me the total number of vehicles in this photo? <image>\n<|assistant|>\n"

image = Image.open(image_path).convert("RGB")
inputs = processor(images=image, text=prompt, return_tensors="pt", padding=True).to(model.device)

with torch.no_grad():
    generated_ids = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=128,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        eos_token_id=tokenizer.eos_token_id
    )

output = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print("Kết quả mô hình:")
print(output)